# Importation des données
[To Do : continuer de décrire origine et format des données]

Les données sont localisées dans des fichiers XML, chaque fichier correspondant à une demande de brevets. Ces fichiers sont directement téléchargeables depuis le serveur FTP de l'INPI, accessible via FileZilla.

Pour rendre notre projet reproductible au maximum, bien que les données ne puissent pas être directement téléchargées par un utilisateur non autorisé par l'INPI, nous avons stocké les fichiers .zip tels que nous les avons initialement téléchargés dans le bucket diffusion de l'un de nous. 

Concernant leur format, ces fichiers compressés contiennent chacun les informations sur toutes les demandes de brevets déposés auprès de l'INPI chaque semaine. Il y a donc 52 dossiers par année, un pour chaque semaine. Dans ces dossiers, se trouvent des informations dont nous n'avons pas besoin (ex: des schémas d'invention). Nous n'utiliserons que les données présentes dans des fichiers XML, chacun de ces fichiers contenant de nombreuses informations sur une demande de brevet déposée dans la semaine concernée.

Le code suivant se charge de décompresser ces fichiers, d'identifier les fichiers XML au sein de leurs arborescences et d'en extraire toutes les informations.

In [1]:
import os
import s3fs
import pandas as pd

from scripts.importation import process_all_years_s3

In [2]:
# Connexion au bucket S3
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

ROOT_S3_PATH = "mvallat/diffusion/projet"

In [3]:
# NB : ce code tourne +7min
data_brevets = process_all_years_s3(fs, ROOT_S3_PATH)
data_brevets.head(2)

/home/onyxia/work/Projet-Python/scripts/importation.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_year["year"] = year_name
/home/onyxia/work/Projet-Python/scripts/importation.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_year["year"] = year_name


⚠️ Erreur XML dans mvallat/diffusion/projet/2022/FR_FRNEWST36_2022_25.zip/FR_FRNEWST36_2022_25/doc/3118340.xml: no element found: line 1, column 0


/home/onyxia/work/Projet-Python/scripts/importation.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_year["year"] = year_name
/home/onyxia/work/Projet-Python/scripts/importation.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_year["year"] = year_name
/home/onyxia/work/Projet-Python/scripts/importation.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) i

,doc-number,kind,country,status,publication_country,publication_doc-number,publication_date,publication_bopinum,publication_nature,invention-title,...,citation_3_country,citation_3_doc-number,citation_3_date,year,application_country,application_doc-number,application_date,last-fee-payement,next-fee-payement,date-search-completed
0,3038352,A1,FR,PUBDEM,FR,3038352,20170106,2017-01,Brevet,ENSEMBLE POUR PANNEAU DE PORTE ET PANNEAU ASSOCIE,...,NA,NA,NA,2017,NaN,NaN,NaN,NaN,NaN,NaN
1,3038278,A1,FR,PUBDEM,FR,3038278,20170106,2017-01,Brevet,VEHICULE HYBRIDE COMPRENANT UNE SUPPRESSION DE...,...,NA,NA,NA,2017,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Exportation bucket S3
MY_BUCKET = "mvallat/diffusion"
FILE_PATH_OUT_S3 = f"{MY_BUCKET}/projet/data_brevets.parquet"

with fs.open(FILE_PATH_OUT_S3, 'wb') as file_out:
    data_brevets.to_parquet(file_out)

Notre base de données est maintenant stockée dans une bucket diffusion sur le ssp cloud, ce qui nous permet de ne pas avoir à repartir des fichiers compressés initiaux. 

**Pour importer directement les données sans passer par les étapes précédentes, on pourra utiliser le code suivant :**

In [5]:
# Importation bucket S3
MY_BUCKET = "mvallat/diffusion"
FILE_PATH_S3 = f"{MY_BUCKET}/projet/data_brevets.parquet"

with fs.open(FILE_PATH_S3, 'rb') as file_in:
    data_brevets = pd.read_parquet(file_in)

data_brevets.head(10)

,doc-number,kind,country,status,publication_country,publication_doc-number,publication_date,publication_bopinum,publication_nature,invention-title,...,citation_3_country,citation_3_doc-number,citation_3_date,year,application_country,application_doc-number,application_date,last-fee-payement,next-fee-payement,date-search-completed
0,3038352,A1,FR,PUBDEM,FR,3038352,20170106,2017-01,Brevet,ENSEMBLE POUR PANNEAU DE PORTE ET PANNEAU ASSOCIE,...,NA,NA,NA,2017,None,None,None,None,None,None
1,3038278,A1,FR,PUBDEM,FR,3038278,20170106,2017-01,Brevet,VEHICULE HYBRIDE COMPRENANT UNE SUPPRESSION DE...,...,NA,NA,NA,2017,None,None,None,None,None,None
2,3038409,A1,FR,PUBDEM,FR,3038409,20170106,2017-01,Brevet,PROCEDE DE CONCEPTION ASSISTEE PAR ORDINATEUR ...,...,NA,NA,NA,2017,None,None,None,None,None,None
3,3038425,A1,FR,PUBDEM,FR,3038425,20170106,2017-01,Brevet,DOCUMENT ELECTRONIQUE TEL QU'UNE CARTE A PUCE ...,...,NA,NA,NA,2017,None,None,None,None,None,None
4,3038316,A1,FR,PUBDEM,FR,3038316,20170106,2017-01,Brevet,PROCEDE DE SYNTHESE EN CONTINU D'UN ELASTOMERE...,...,NA,NA,NA,2017,None,None,None,None,None,None
5,3038399,A1,FR,PUBDEM,FR,3038399,20170106,2017-01,Brevet,PROCEDE ET DISPOSITIF DE COMMANDE D'UNE CONSIG...,...,NA,NA,NA,2017,None,None,None,None,None,None
6,3038381,A1,FR,PUBDEM,FR,3038381,20170106,2017-01,Brevet,DISPOSITIF CAPTEUR PREVU POUR ETRE EMBARQUE DE...,...,NA,NA,NA,2017,None,None,None,None,None,None
7,3038328,A1,FR,PUBDEM,FR,3038328,20170106,2017-01,Brevet,VEHICULE FERROVIAIRE POUR LE NETTOYAGE DES FIL...,...,NA,NA,NA,2017,None,None,None,None,None,None
8,3038251,A1,FR,PUBDEM,FR,3038251,20170106,2017-01,Brevet,PLANCHER PVC EN FIBRES COUPEES ET PROCEDE DE F...,...,NA,NA,NA,2017,None,None,None,None,None,None
9,3038374,A1,FR,PUBDEM,FR,3038374,20170106,2017-01,Brevet,INSTALLATION THERMIQUE DE SECHAGE DE MATIERE P...,...,NA,NA,NA,2017,None,None,None,None,None,None
